In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from huggingface_hub import notebook_login
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import os
import sys

In [2]:
torch.cuda.is_available()

True

In [3]:
#now we load in the pdfs
document = []
for file in os.listdir("docs"):
    print(file)
    if file.endswith(".pdf"):
        pdf_path = ".\\docs\\"+file
        loader = PyPDFLoader(pdf_path)
        document.extend(loader.load())


ip11pmaxspecs.pdf
iphone13pm.pdf
iphoneSEspecs.pdf


In [4]:
print(document)

[Document(page_content='IPhone  11 Pro Max Specifications  \nNETWORK  Technology  GSM  / CDMA  / HSPA  / EVDO  / LTE    \nLAUNCH  Announced  2019, September  \nStatus  Coming soon. Exp. release 2019, September  \nBODY  Dimensions  158 x 77.8 x 8.1 mm (6.22 x 3.06 x 0.32 in)  \nWeight  226 g (7.97 oz)  \nBuild  Front/back glass, stainless steel frame  \nSIM  Single SIM (Nano -SIM and/or Electronic SIM card) or Dual SIM (Nano -SIM, dual \nstand -by) - for China  \n  IP68 dust/water resistant (up to 4m for 30 mins)  \nApple Pay (Visa, MasterCard, AMEX certified)  \nDISPLAY  Type  Super Retina XDR OLED capacitive touchscreen, 16M colors  \nSize  6.5 inches, 102.9 cm2 (~83.7% screen -to-body ratio)  \nResolution  1242 x 2688 pixels, 19.5:9 ratio (~458 ppi density)  \nProtection  Scratch -resistant glass, oleophobic coating  \n  800 nits  \nDolby Vision \nHDR10  \nWide color gamut  \nTrue-tone \n120 Hz touch -sensing  \nPLATFORM  OS  iOS 13  \nChipset  Apple A13 Bionic (7 nm+)  \nMEMORY  Car

In [5]:
document_splitta=CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
document_chunks=document_splitta.split_documents(document)
len(document_chunks)

6

In [6]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\Preetham\Desktop\SpecSheet\spec_env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
vectordb = Chroma.from_documents(document_chunks, embedding=embeddings, persist_directory='.\\data')

In [8]:
vectordb.persist()

C:\Users\Preetham\Desktop\SpecSheet\spec_env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [9]:
notebook_login()

In [11]:
model_id = "bartowski/Meta-Llama-3-8B-Instruct-GGUF-hf"
filename = "Meta-Llama-3-8B-Instruct-Q8_0.gguf"
tokenizer = AutoTokenizer.from_pretrained(model_id, gguf_file=filename,device_type='cuda')
model = AutoModelForCausalLM.from_pretrained(model_id,gguf_file=filename, device_map='cuda')

OSError: bartowski/Meta-Llama-3-8B-Instruct-GGUF-hf is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
pipe=pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                torch_dtype=torch.bfloat16,
                device_map='cuda',
                max_new_tokens=512,
                min_new_tokens=-1,
                top_k=30)

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [ ]:
llm

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [ ]:
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm, retriever=vectordb.as_retriever(search_kwargs={'k':6}),verbose=False,memory=memory)

In [ ]:
result=pdf_qa({"question":"What iPhone has the A15 Bionic chip?"})

In [ ]:
print(result)

In [ ]:
print(result['answer'])

In [ ]:
while True:
    query=input(f"Prompt:")
    if query == "exit" or query == "quit" or query == "q" or query == "f":
        print('Exiting')
        sys.exit()
    if query == '':
        continue
    result = pdf_qa({"question": query})
    print(f"Answer: " + result["answer"])
